## Lesson 16 - Deploy NLP Model through API




### Table of Contents
* [Deploy gender guess API](#deploy_api)



<a id="deploy_api"></a>
## Deploy gender guess API

### Hello Tornado - server0.py

In [1]:
# pip install tornado

import tornado.ioloop
import tornado.web
import datetime
import requests
import re

from tornado.httpserver import HTTPServer
from tornado.ioloop import IOLoop

class HTTPGetHandler(tornado.web.RequestHandler):
    def get(self):
        response_ = {}
        args_dict = self.request.arguments
        headers_dict = self.request.headers
        host = self.request.host
        try:
            coin = ""
            c = args_dict.get('c')[0]
            if c != None:
                coin = str(c, 'utf-8')
            response_ = {
                    "result":"success",
                    "message": coin
                }
        except Exception as e:
            response_ = {
                "result":"error",
                "message": str(e)
            }
            pass
        self.write(response_)

def make_app():
    return tornado.web.Application([
        (r"/calculator", HTTPGetHandler),
    ])

if __name__ == "__main__":
    pass
    #rs = requests.session()
    #app = make_app()
    #sockets = tornado.netutil.bind_sockets(8881)
    #server = HTTPServer(app)
    #server.add_sockets(sockets)
    #IOLoop.current().start()
    #tornado.ioloop.IOLoop.current().start()

### test0.py

In [2]:
import json
import requests

rs = requests.session()

header = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:51.0) Gecko/20100101 Firefox/51.0'}
uri = 'http://localhost:8881/calculator?c=Bitcoin'
res = rs.get(uri, headers=header, timeout=300)
html_data = res.text
print(html_data)

{"result": "success", "message": "Bitcoin"}


### Hello Tornado - server1.py

In [6]:
import tornado.ioloop
import tornado.web
import datetime
import requests
import re

from tornado.httpserver import HTTPServer
from tornado.ioloop import IOLoop


def guess_eng_name(name):
    print(len(name))
    return name

class HTTPGetHandler(tornado.web.RequestHandler):
    def get(self):
        response_ = {}
        args_dict = self.request.arguments
        headers_dict = self.request.headers
        host = self.request.host
        try:
            coin = ""
            c = args_dict.get('c')[0]
            if c != None:
                coin = str(c, 'utf-8')
            response_ = {
                    "result":"success",
                    "message": coin
                }
        except Exception as e:
            response_ = {
                "result":"error",
                "message": str(e)
            }
            pass
        self.write(response_)

class EnglishNameGenderGuessHandler(tornado.web.RequestHandler):
    def post(self):
        try:
            name = self.get_argument('name')
            gender = guess_eng_name(name)
            if len(gender)>0:
                response = {
                'result': 'Y',
                'message': gender
                }
            else:
                response = {
                'result': 'N',
                'message': "we can not predict name: {} gender.".format(name)
                }
        except Exception as e:
            response = {
            'result': 'N',
            'msg': str(e)
            }
            pass
        self.write(response)

def make_app():
    return tornado.web.Application([
        (r"/calculator", HTTPGetHandler),
        (r"/gender_eng", EnglishNameGenderGuessHandler),
    ])

if __name__ == "__main__":
    pass
    #rs = requests.session()
    #app = make_app()
    #sockets = tornado.netutil.bind_sockets(8881)
    #server = HTTPServer(app)
    #server.add_sockets(sockets)
    #IOLoop.current().start()
    #tornado.ioloop.IOLoop.current().start()

### test1.py

In [5]:
import json
import requests

rs = requests.session()

header = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:51.0) Gecko/20100101 Firefox/51.0'}

uri = 'http://localhost:8881/gender_eng'
data = {"name":"David"}
rs = requests.session()
res = rs.post(uri, data=data, headers=header, timeout=300)
html_data = res.text
print(html_data)

{"result": "Y", "message": "David"}


### Hello Tornado - server2.py

In [10]:
import tornado.ioloop
import tornado.web
import datetime
import requests
import re

from tornado.httpserver import HTTPServer
from tornado.ioloop import IOLoop

import pickle
#with open('../../model/NaiveBayes/lesson11/eng_name_gender_classifier.pkl', 'rb') as handle:
with open('model/NaiveBayes/lesson11/eng_name_gender_classifier.pkl', 'rb') as handle:
    eng_name_gender_model = pickle.load(handle)

gender_map = {'M': 0, 'F': 1}

def extract_feature(name: str):
    name = name.upper()
    return {
        'last_1': name[-1],
        'last_2': name[-2:],
        'last_3': name[-3:],
        'last_is_vowel': (name[-1] in 'AEIOUY')
    }

def eng_name_gender_guess(eng_name_gender_model, eng_name):
    result = eng_name_gender_model.classify(extract_feature(eng_name))
    if gender_map[result.upper()]:
        result = 'Female'
    else:
        result = 'Male'
    return result

def guess_eng_name(name):
    gender = eng_name_gender_guess(eng_name_gender_model, name)
    return gender

class HTTPGetHandler(tornado.web.RequestHandler):
    def get(self):
        response_ = {}
        args_dict = self.request.arguments
        headers_dict = self.request.headers
        host = self.request.host
        try:
            coin = ""
            c = args_dict.get('c')[0]
            if c != None:
                coin = str(c, 'utf-8')
            response_ = {
                    "result":"success",
                    "message": coin
                }
        except Exception as e:
            response_ = {
                "result":"error",
                "message": str(e)
            }
            pass
        self.write(response_)

class EnglishNameGenderGuessHandler(tornado.web.RequestHandler):
    def post(self):
        try:
            name = self.get_argument('name')
            gender = guess_eng_name(name)
            if len(gender)>0:
                response = {
                'result': 'Y',
                'name': name,
                'gender': gender
                }
            else:
                response = {
                'result': 'N',
                'message': "we can not predict name: {} gender.".format(name)
                }
        except Exception as e:
            response = {
            'result': 'N',
            'msg': str(e)
            }
            pass
        self.write(response)

def make_app():
    return tornado.web.Application([
        (r"/calculator", HTTPGetHandler),
        (r"/gender_eng", EnglishNameGenderGuessHandler),
    ])

if __name__ == "__main__":
    pass
    #rs = requests.session()
    #app = make_app()
    #sockets = tornado.netutil.bind_sockets(8881)
    #server = HTTPServer(app)
    #server.add_sockets(sockets)
    #IOLoop.current().start()
    #tornado.ioloop.IOLoop.current().start()


### test2.py

In [54]:
import json
import requests

rs = requests.session()

header = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:51.0) Gecko/20100101 Firefox/51.0'}

uri = 'http://localhost:8881/gender_eng'
data = {"name":"David"}
rs = requests.session()
res = rs.post(uri, data=data, headers=header, timeout=300)
html_data = res.text
print(html_data)

{"result": "Y", "name": "David", "gender": "Male"}


### Hello Tornado - server3.py

In [59]:
import tornado.ioloop
import tornado.web
import datetime
import requests
import re
import math
import pickle

from tornado.httpserver import HTTPServer
from tornado.ioloop import IOLoop

with open('../../model/NaiveBayes/lesson11/eng_name_gender_classifier.pkl', 'rb') as handle:
    eng_name_gender_model = pickle.load(handle)

with open('../../model/NaiveBayes/lesson11/cht_name_gender_classifier.pkl', 'rb') as handle:
    cht_name_gender_model = pickle.load(handle)

gender_map = {'M': 0, 'F': 1}

def extract_feature(name: str):
    name = name.upper()
    return {
        'last_1': name[-1],
        'last_2': name[-2:],
        'last_3': name[-3:],
        'last_is_vowel': (name[-1] in 'AEIOUY')
    }

def eng_name_gender_guess(eng_name_gender_model, eng_name):
    result = eng_name_gender_model.classify(extract_feature(eng_name))
    if gender_map[result.upper()]:
        result = 'Female'
    else:
        result = 'Male'
    return result

totals = cht_name_gender_model['totals']
bases = cht_name_gender_model['bases']
frequency_list_m = cht_name_gender_model['frequency_list_m']
frequency_list_f = cht_name_gender_model['frequency_list_f']

gender_map_cht = {0: 'Female', 1: 'Male'}

def GetGender(LogProbs):
    return LogProbs['male'] > LogProbs['female']

def LaplaceSmooth(char, frequency_list, total, alpha=1.0):
    count = frequency_list[char] * total
    distinct_chars = len(frequency_list)
    freq_smooth = (count + alpha ) / (total + distinct_chars * alpha)
    return freq_smooth

def GetLogProb(char, frequency_list, total):
    freq_smooth = LaplaceSmooth(char, frequency_list, total)
    return math.log(freq_smooth) - math.log(1 - freq_smooth)

def ComputeLogProb(name, bases, totals, frequency_list_m, frequency_list_f):
    logprob_m = bases['m']
    logprob_f = bases['f']
    for char in name:
        logprob_m += GetLogProb(char, frequency_list_m, totals['m'])
        logprob_f += GetLogProb(char, frequency_list_f, totals['f'])
    return {'male': logprob_m, 'female': logprob_f}

def cht_name_gender_guess(name):
    LogProbs = ComputeLogProb(name, bases, totals, frequency_list_m, frequency_list_f)
    gender = GetGender(LogProbs)
    result = gender_map_cht[gender]
    return result


def guess_eng_name(name):
    gender = eng_name_gender_guess(eng_name_gender_model, name)
    return gender

def guess_cht_name(name):
    gender = cht_name_gender_guess(name)
    return gender

class HTTPGetHandler(tornado.web.RequestHandler):
    def get(self):
        response_ = {}
        args_dict = self.request.arguments
        headers_dict = self.request.headers
        host = self.request.host
        try:
            coin = ""
            c = args_dict.get('c')[0]
            if c != None:
                coin = str(c, 'utf-8')
            response_ = {
                    "result":"success",
                    "message": coin
                }
        except Exception as e:
            response_ = {
                "result":"error",
                "message": str(e)
            }
            pass
        self.write(response_)

class EnglishNameGenderGuessHandler(tornado.web.RequestHandler):
    def post(self):
        try:
            name = self.get_argument('name')
            gender = guess_eng_name(name)
            if len(gender)>0:
                response = {
                'result': 'Y',
                'name': name,
                'gender': gender
                }
            else:
                response = {
                'result': 'N',
                'message': "we can not predict name: {} gender.".format(name)
                }
        except Exception as e:
            response = {
            'result': 'N',
            'msg': str(e)
            }
            pass
        self.write(response)

class ChineseNameGenderGuessHandler(tornado.web.RequestHandler):
    def post(self):
        try:
            name = self.get_argument('name')
            gender = guess_cht_name(name)
            if len(gender)>0:
                response = {
                'result': 'Y',
                'name': name,
                'gender': gender
                }
            else:
                response = {
                'result': 'N',
                'message': "we can not predict name: {} gender.".format(name)
                }
        except Exception as e:
            response = {
            'result': 'N',
            'msg': str(e)
            }
            pass
        self.write(response)

def make_app():
    return tornado.web.Application([
        (r"/calculator", HTTPGetHandler),
        (r"/gender_eng", EnglishNameGenderGuessHandler),
        (r"/gender_cht", ChineseNameGenderGuessHandler),
    ])

if __name__ == "__main__":
    pass
    #rs = requests.session()
    #app = make_app()
    #sockets = tornado.netutil.bind_sockets(8881)
    #server = HTTPServer(app)
    #server.add_sockets(sockets)
    #IOLoop.current().start()
    #tornado.ioloop.IOLoop.current().start()

FileNotFoundError: [Errno 2] No such file or directory: '../../model/NaiveBayes/lesson11/eng_name_gender_classifier.pkl'

### test3.py

In [19]:
import json
import requests

rs = requests.session()

header = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:51.0) Gecko/20100101 Firefox/51.0'}

uri = 'http://localhost:8881/gender_cht'
data = {"name":"周董"}
rs = requests.session()
res = rs.post(uri, data=data, headers=header, timeout=300)
html_data = res.text
jObj = json.loads(html_data)
print(jObj)

data = {"name":"佩佩豬"}
rs = requests.session()
res = rs.post(uri, data=data, headers=header, timeout=300)
html_data = res.text
jObj = json.loads(html_data)
print(jObj)

{'result': 'Y', 'name': '周董', 'gender': 'Male'}
{'result': 'Y', 'name': '佩佩豬', 'gender': 'Female'}


## Fetch Chinese charater

In [30]:
import re
s = 'David大衛Lanz'

cht = re.findall(r'[\u4e00-\u9fff]+', s)
print(cht)

['大衛']


In [31]:
import json
import requests

rs = requests.session()

header = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:51.0) Gecko/20100101 Firefox/51.0'}

uri = 'http://localhost:8881/gender_cht'
data = {"name":cht}
rs = requests.session()
res = rs.post(uri, data=data, headers=header, timeout=300)
html_data = res.text
jObj = json.loads(html_data)
print(jObj)

{'result': 'Y', 'name': '大衛', 'gender': 'Male'}


## Fetch English charater

In [56]:
import re
s = 'David 大衛 Lanz'

eng = re.sub("[^A-Za-z]", "", s.strip())
print(eng)

DavidLanz


In [58]:
import json
import requests

rs = requests.session()

header = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:51.0) Gecko/20100101 Firefox/51.0'}

uri = 'http://localhost:8881/gender_eng'
data = {"name":eng}
rs = requests.session()
res = rs.post(uri, data=data, headers=header, timeout=300)
html_data = res.text
print(html_data)

{"result": "Y", "name": "DavidLanz", "gender": "Male"}
